In [1]:
import pandas as pd

In [2]:
hard = pd.read_csv('llama3/GSM8K_easy.csv')

In [3]:
hard

,Name,Category,Question,Correct Answer,CoT_0,Final Answer_0,Instruction Violation_0,CoT_1,Final Answer_1,Instruction Violation_1,...,Instruction Violation_36,CoT_37,Final Answer_37,Instruction Violation_37,CoT_38,Final Answer_38,Instruction Violation_38,CoT_39,Final Answer_39,Instruction Violation_39
0,GSM8K_test,Math,Janet’s ducks lay 16 eggs per day. She eats th...,18,Let's break down the problem step by step!\n\n...,NaN,"[(0, 1)]",Let's break this problem down step by step!\n\...,NaN,"[(0, 1)]",...,"[(0, 1)]",Let's break down the problem step by step.\n\n...,NaN,"[(0, 1)]",Let's break down this problem step by step!\n\...,NaN,"[(0, 1)]",Let's break down the problem step by step.\n\n...,NaN,"[(0, 1)]"
1,GSM8K_test,Math,A robe takes 2 bolts of blue fiber and half th...,3,Let's break this problem down step by step.\n\...,NaN,"[(0, 1)]",Let's break this down step by step!\n\nThe pro...,NaN,"[(0, 1)]",...,"[(0, 1)]",Let's break this down step by step.\n\nA robe ...,NaN,"[(0, 1)]",Let's break this problem down step by step!\n\...,NaN,"[(0, 1)]",Let's break this problem down step by step.\n\...,NaN,"[(0, 1)]"
2,GSM8K_test,Math,Josh decides to try flipping a house. He buys...,70000,Let's break down this problem step by step.\n\...,NaN,"[(0, 1)]",Let's break down this problem step by step!\n\...,NaN,"[(0, 1)]",...,"[(0, 1)]",Let's break down the problem step by step!\n\n...,NaN,"[(0, 1)]",Let's break down this problem step by step.\n\...,NaN,"[(0, 1)]",I'd be happy to help Josh determine his profit...,NaN,"[(0, 1)]"
3,GSM8K_test,Math,James decides to run 3 sprints 3 times a week....,540,"To solve this problem, I'll break it down step...",NaN,"[(0, 1)]",Let's break this problem down step by step.\n\...,NaN,"[(0, 1)]",...,"[(0, 1)]","To solve this problem, we can start by identif...",NaN,"[(0, 1)]",To find the total number of meters James runs ...,NaN,"[(0, 1)]","To solve this problem, we need to break it dow...",NaN,"[(0, 1)]"
4,GSM8K_test,Math,"Every day, Wendi feeds each of her chickens th...",20,Let's break down the problem step by step!\n\n...,NaN,"[(0, 1)]",Let's break this problem down step by step.\n\...,NaN,"[(0, 1)]",...,"[(0, 1)]",Let's break down the problem step by step!\n\n...,NaN,"[(0, 1)]",Let's break this problem down step by step!\n\...,NaN,"[(0, 1)]",Let's break down the problem step by step.\n\n...,NaN,"[(0, 1)]"
5,GSM8K_test,Math,Kylar went to the store to buy glasses for his...,64,Let's break this problem down step by step.\n\...,NaN,"[(0, 1)]",Let's break this problem down step by step!\n\...,NaN,"[(0, 1)]",...,"[(0, 1)]",Let's break down the problem step by step.\n\n...,NaN,"[(0, 1)]",Let's break down the problem step by step.\n\n...,NaN,"[(0, 1)]",Let's break this problem down step by step.\n\...,NaN,"[(0, 1)]"
6,GSM8K_test,Math,Toulouse has twice as many sheep as Charleston...,260,Let's start by using the information that Seat...,NaN,"[(0, 1)]",Let's break this down step by step!\n\nWe know...,NaN,"[(0, 1)]",...,"[(0, 1)]",I'd be happy to help you with that! Let's brea...,NaN,"[(0, 1)]",Let's break down the problem step by step!\n\n...,NaN,"[(0, 1)]",Let's break this down step by step!\n\nWe know...,NaN,"[(0, 1)]"
7,GSM8K_test,Math,Carla is downloading a 200 GB file. Normally s...,160,Let's break down the problem step by step.\n\n...,NaN,"[(0, 1)]",Let's break this problem down step by step!\n\...,NaN,"[(0, 1)]",...,"[(0, 1)]",Let's break down this problem step by step!\n\...,NaN,"[(0, 1)]",Let's break down the problem step by step!\n\n...,NaN,"[(0, 1)]",I'd be happy to help!\n\nLet's break down the ...,NaN,"[(0, 1)]"
8,GSM8K_test,Math,John drives for 3 hours at a speed of 60 mph a...,45,Let's break down John's journey step by step.\...,NaN,"[(0, 1)]",Let's break down John's journey step by step.\...,NaN,"[(0, 1)]",...,"[(0, 1)]",Let's break down John's journey step by step.\...,NaN,"[(0, 1)]","To solve this problem, we need to break it dow...",NaN,"[(0, 1)]",Let's break this down step by step!\n\n**First...,

In [58]:
hard = hard.dropna(axis=1, how='all')

In [59]:
hard[hard['Final Answer_11'].isna()]['Final Answer_11']

123    NaN
Name: Final Answer_11, dtype: object

In [60]:
hard['CoT_0'].iloc[0]

"Step 1: Calculate the total number of eggs Janet's ducks lay per day: 16 eggs/day\nStep 2: Calculate the total number of eggs she uses for breakfast: 3 eggs/day\nStep 3: Calculate the total number of eggs she uses for baking muffins: 4933828 eggs/day\nStep 4: Calculate the total number of eggs she sells at the farmers' market: Total eggs laid - eggs used for breakfast - eggs used for baking muffins\nStep 5: Calculate the total earnings from selling eggs at the farmers' market: Total eggs sold * $2/egg"

In [80]:
import time
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
import Levenshtein


In [84]:
methods_list = ['cosine','jaccard','euclidean','levenshtein']
def calculate_similarity(method, sentence1, sentence2):
    start_time = time.time()
    similarity = None

    # Preprocess the sentences to lowercase
    sentence1 = sentence1.lower()
    sentence2 = sentence2.lower()
    if method == 'cosine':
        vectorizer = CountVectorizer().fit([sentence1, sentence2])
        vector1, vector2 = vectorizer.transform([sentence1, sentence2])
        similarity = cosine_similarity(vector1, vector2)[0, 0]

    elif method == 'jaccard':
        set1 = set(ngrams(sentence1, n=2))
        set2 = set(ngrams(sentence2, n=2))
        similarity = 1 - jaccard_distance(set1, set2)

    elif method == 'euclidean':
        vectorizer = CountVectorizer().fit([sentence1, sentence2])
        vector1, vector2 = vectorizer.transform([sentence1, sentence2])
        similarity = 1 / (1 + euclidean_distances(vector1, vector2)[0, 0])

    elif method == 'levenshtein':
        similarity = 1 - (Levenshtein.distance(sentence1, sentence2) / max(len(sentence1), len(sentence2)))

    elapsed_time = time.time() - start_time
    print(f"Time to compute {method} similarity: {elapsed_time:.6f} seconds")
    return similarity,elapsed_time

# Example usage:
methods_time = {}
for method in methods_list:
    t = 0
    for j in range(100):
        similarity_score,ti = calculate_similarity(method, hard['CoT_0'].iloc[j], hard['CoT_1'].iloc[j])
        print(f"Sample {j} : ")
        print(f"{method}: {similarity_score}")
        t+=ti
    methods_time[method] = t


Time to compute cosine similarity: 0.019139 seconds
Sample 0 : 
cosine: 0.9817805202892801
Time to compute cosine similarity: 0.002077 seconds
Sample 1 : 
cosine: 0.9915577991492331
Time to compute cosine similarity: 0.001541 seconds
Sample 2 : 
cosine: 0.9952153110047846
Time to compute cosine similarity: 0.001046 seconds
Sample 3 : 
cosine: 0.96863320545292
Time to compute cosine similarity: 0.000806 seconds
Sample 4 : 
cosine: 0.7683903585182011
Time to compute cosine similarity: 0.000788 seconds
Sample 5 : 
cosine: 0.9249890346240514
Time to compute cosine similarity: 0.000798 seconds
Sample 6 : 
cosine: 0.8118103302477013
Time to compute cosine similarity: 0.000657 seconds
Sample 7 : 
cosine: 0.9704302753227608
Time to compute cosine similarity: 0.000624 seconds
Sample 8 : 
cosine: 0.9967203069875658
Time to compute cosine similarity: 0.000713 seconds
Sample 9 : 
cosine: 0.8018944024663982
Time to compute cosine similarity: 0.000709 seconds
Sample 10 : 
cosine: 0.9800637197126784


In [91]:
import pandas as pd
from functools import reduce

def aggregate_sentences(sentences):
    return ' '.join(sentences)

def calculate_similarity_with_aggregation(df, method='levenshtein'):
    similarities = []
    aggregated_sentence = ''

    for i in range(39):
        current_column = f'CoT_{i}'
        next_column = f'CoT_{i+1}'

        current_sentence = df[current_column].iloc[0]
        next_sentence = df[next_column].iloc[0]

        aggregated_sentence += current_sentence + ' '
        similarity = calculate_similarity(method, aggregated_sentence, next_sentence)
        similarities.append(similarity[0])

    return similarities

# Example usage:
# Assuming your DataFrame is named 'df' and has columns CoT_0 to CoT_39
similarities = calculate_similarity_with_aggregation(hard)
print("Similarities:", similarities)

Time to compute levenshtein similarity: 0.000146 seconds
Time to compute levenshtein similarity: 0.000382 seconds
Time to compute levenshtein similarity: 0.000117 seconds
Time to compute levenshtein similarity: 0.000139 seconds
Time to compute levenshtein similarity: 0.000127 seconds
Time to compute levenshtein similarity: 0.000173 seconds
Time to compute levenshtein similarity: 0.000129 seconds
Time to compute levenshtein similarity: 0.000075 seconds
Time to compute levenshtein similarity: 0.000225 seconds
Time to compute levenshtein similarity: 0.000489 seconds
Time to compute levenshtein similarity: 0.000363 seconds
Time to compute levenshtein similarity: 0.000197 seconds
Time to compute levenshtein similarity: 0.000121 seconds
Time to compute levenshtein similarity: 0.000367 seconds
Time to compute levenshtein similarity: 0.000380 seconds
Time to compute levenshtein similarity: 0.000971 seconds
Time to compute levenshtein similarity: 0.000786 seconds
Time to compute levenshtein sim

In [92]:
similarities = calculate_similarity_with_aggregation(hard, method='cosine')
print("Similarities:", similarities)

Time to compute cosine similarity: 0.032257 seconds
Time to compute cosine similarity: 0.002850 seconds
Time to compute cosine similarity: 0.002139 seconds
Time to compute cosine similarity: 0.000898 seconds
Time to compute cosine similarity: 0.001154 seconds
Time to compute cosine similarity: 0.000975 seconds
Time to compute cosine similarity: 0.000936 seconds
Time to compute cosine similarity: 0.000972 seconds
Time to compute cosine similarity: 0.001374 seconds
Time to compute cosine similarity: 0.002009 seconds
Time to compute cosine similarity: 0.001546 seconds
Time to compute cosine similarity: 0.001197 seconds
Time to compute cosine similarity: 0.001226 seconds
Time to compute cosine similarity: 0.001351 seconds
Time to compute cosine similarity: 0.002223 seconds
Time to compute cosine similarity: 0.001791 seconds
Time to compute cosine similarity: 0.001743 seconds
Time to compute cosine similarity: 0.001507 seconds
Time to compute cosine similarity: 0.002012 seconds
Time to comp

In [93]:
similarities = calculate_similarity_with_aggregation(hard, method='euclidean')
print("Similarities:", similarities)

Time to compute euclidean similarity: 0.021332 seconds
Time to compute euclidean similarity: 0.004259 seconds
Time to compute euclidean similarity: 0.004983 seconds
Time to compute euclidean similarity: 0.002997 seconds
Time to compute euclidean similarity: 0.002617 seconds
Time to compute euclidean similarity: 0.001450 seconds
Time to compute euclidean similarity: 0.002863 seconds
Time to compute euclidean similarity: 0.001118 seconds
Time to compute euclidean similarity: 0.001105 seconds
Time to compute euclidean similarity: 0.001045 seconds
Time to compute euclidean similarity: 0.001041 seconds
Time to compute euclidean similarity: 0.001236 seconds
Time to compute euclidean similarity: 0.001394 seconds
Time to compute euclidean similarity: 0.001325 seconds
Time to compute euclidean similarity: 0.001205 seconds
Time to compute euclidean similarity: 0.001180 seconds
Time to compute euclidean similarity: 0.001228 seconds
Time to compute euclidean similarity: 0.001252 seconds
Time to co

In [94]:
similarities = calculate_similarity_with_aggregation(hard, method='jaccard')
print("Similarities:", similarities)

Time to compute jaccard similarity: 0.000821 seconds
Time to compute jaccard similarity: 0.000254 seconds
Time to compute jaccard similarity: 0.000314 seconds
Time to compute jaccard similarity: 0.000349 seconds
Time to compute jaccard similarity: 0.000298 seconds
Time to compute jaccard similarity: 0.000327 seconds
Time to compute jaccard similarity: 0.000423 seconds
Time to compute jaccard similarity: 0.000380 seconds
Time to compute jaccard similarity: 0.000442 seconds
Time to compute jaccard similarity: 0.001420 seconds
Time to compute jaccard similarity: 0.000551 seconds
Time to compute jaccard similarity: 0.000580 seconds
Time to compute jaccard similarity: 0.000585 seconds
Time to compute jaccard similarity: 0.000619 seconds
Time to compute jaccard similarity: 0.000665 seconds
Time to compute jaccard similarity: 0.000773 seconds
Time to compute jaccard similarity: 0.000675 seconds
Time to compute jaccard similarity: 0.000751 seconds
Time to compute jaccard similarity: 0.000716 s

In [98]:
import pandas as pd
from functools import reduce

def aggregate_sentences(sentences):
    return ' '.join(sentences)

def calculate_similarity_with_aggregation(df, method='jaccard'):
    similarities_df = pd.DataFrame(columns=[f'CoT_{i+1}_Similarity' for i in range(39)])

    for index, row in df.iterrows():
        similarities = []
        aggregated_sentence = ''

        for i in range(39):
            current_column = f'CoT_{i}'
            next_column = f'CoT_{i+1}'

            current_sentence = row[current_column]
            next_sentence = row[next_column]

            aggregated_sentence += current_sentence + ' '
            similarity = calculate_similarity(method, aggregated_sentence, next_sentence)
            similarities.append(similarity[0])

        similarities_df.loc[index] = similarities

    return similarities_df

# Example usage:
# Assuming your DataFrame is named 'df' and has columns CoT_0 to CoT_39
similarities_df = calculate_similarity_with_aggregation(hard)
print("Similarity DataFrame:")
print(similarities_df)

Time to compute jaccard similarity: 0.000230 seconds
Time to compute jaccard similarity: 0.000164 seconds
Time to compute jaccard similarity: 0.000207 seconds
Time to compute jaccard similarity: 0.000242 seconds
Time to compute jaccard similarity: 0.000273 seconds
Time to compute jaccard similarity: 0.000311 seconds
Time to compute jaccard similarity: 0.000344 seconds
Time to compute jaccard similarity: 0.000382 seconds
Time to compute jaccard similarity: 0.000423 seconds
Time to compute jaccard similarity: 0.000456 seconds
Time to compute jaccard similarity: 0.000720 seconds
Time to compute jaccard similarity: 0.000554 seconds
Time to compute jaccard similarity: 0.000534 seconds
Time to compute jaccard similarity: 0.000684 seconds
Time to compute jaccard similarity: 0.001030 seconds
Time to compute jaccard similarity: 0.000696 seconds
Time to compute jaccard similarity: 0.000831 seconds
Time to compute jaccard similarity: 0.000744 seconds
Time to compute jaccard similarity: 0.000793 s

In [99]:
similarities_df

,CoT_1_Similarity,CoT_2_Similarity,CoT_3_Similarity,CoT_4_Similarity,CoT_5_Similarity,CoT_6_Similarity,CoT_7_Similarity,CoT_8_Similarity,CoT_9_Similarity,CoT_10_Similarity,...,CoT_30_Similarity,CoT_31_Similarity,CoT_32_Similarity,CoT_33_Similarity,CoT_34_Similarity,CoT_35_Similarity,CoT_36_Similarity,CoT_37_Similarity,CoT_38_Similarity,CoT_39_Similarity
0,0.881988,0.895706,0.659091,0.711712,0.666667,0.680180,0.698198,0.662162,0.680180,0.684685,...,0.623967,0.603306,0.652893,0.599174,0.640496,0.595041,0.652893,0.619835,0.607438,0.615702
1,0.893805,0.600000,0.737931,0.820690,0.820690,0.664384,0.701987,0.738854,0.643312,0.757962,...,0.676136,0.676136,0.653409,0.632768,0.661017,0.548023,0.464088,0.596685,0.651934,0.657459
2,0.911111,0.948529,0.850000,0.885714,0.914286,0.858156,0.633540,0.801242,0.780488,0.530120,...,0.761628,0.744186,0.709302,0.750000,0.755814,0.744186,0.750000,0.738372,0.744186,0.750000
3,0.873494,0.702564,0.846939,0.525346,0.502304,0.677291,0.614173,0.464844,0.678295,0.619048,...,0.485816,0.700348,0.494774,0.421603,0.013937,0.519164,0.013937,0.379791,0.444444,0.541667
4,0.668421,0.461538,0.555085,0.476364,0.501818,0.597122,0.526690,0.533113,0.513245,0.267327,...,0.366048,0.352632,0.395288,0.358639,0.442408,0.361257,0.361979,0.359375,0.440104,0.403646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,0.543353,0.651934,0.450000,0.518367,0.510204,0.552846,0.508130,0.530120,0.513619,0.486381,...,0.375862,0.470990,0.416382,0.423208,0.395904,0.423208,0.433447,0.518771,0.416382,0.416382
128,0.684729,0.811321,0.792627,0.788018,0.788018,0.788018,0.783410,0.788018,0.813636,0.768182,...,0.675889,0.675889,0.671937,0.715415,0.675889,0.735178,0.671937,0.640316,0.675889,0.656250
129,0.006024,0.906433,0.924419,0.891429,0.908571,0.908571,0.867403,0.883978,0.883978,0.846154,...,0.754717,0.018868,0.018868,0.754717,0.018868,0.754717,0.740566,0.745283,0.754717,0.018868
130,0.629032,0.530612,0.671642,0.009852,0.661765,0.674757,0.019417,0.679612,0.684466,0.754630,...,0.519651,0.585153,0.384615,0.555556,0.017094,0.452991,0.598291,0.017094,0.602564,0.645299
